In [1]:
import json, os, cv2
import numpy as np
import xml.etree.ElementTree as ET
from tqdm import tqdm

### 수집한 데이터

In [2]:
data_type = 'collection'
images_folder_path = './raw_data/collection/images/'
json_path = './raw_data/collection/labels/total.json'
save_folder_path = '/generated_data/'

In [3]:
with open(json_path, 'r') as f:
    json_data = json.load(f)

In [4]:
for key in tqdm(list(json_data.keys()), desc=f'generating yolo_format_data of {data_type}'):
    
    img_name = json_data[key]['filename'].split('.')[0]
    img_path = images_folder_path + img_name + '.JPG'
    img = cv2.imread(img_path)
    h_, w_ = img.shape[0], img.shape[1]

    # images 생성
    cv2.imwrite(f'.{save_folder_path}{data_type}/images/train/{img_name}.jpg', img)
    
    # labels 생성
    bbox_regions = json_data[key]['regions']
    f = open(f'.{save_folder_path}{data_type}/labels/train/{img_name}.txt', 'w')
    for bbox_attr in bbox_regions:    
        class_type = 0
        
        
        x = round((bbox_attr['shape_attributes']['x'] + bbox_attr['shape_attributes']['width']//2)/w_, 4)
        y = round((bbox_attr['shape_attributes']['y'] + bbox_attr['shape_attributes']['height']//2)/h_, 4)
        w = round(bbox_attr['shape_attributes']['width']/w_, 4)
        h = round(bbox_attr['shape_attributes']['height']/h_, 4)

        text = '{} {} {} {} {}\n'.format(str(class_type), str(x), str(y), str(w), str(h))
        f.write(text)
    f.close()

    

generating yolo_format_data of collection: 100%|████████████████████████████████████████████| 326/326 [00:40<00:00,  8.14it/s]


### 캐글 데이터

In [5]:
data_type = 'kaggle'
images_folder_path = './raw_data/kaggle/images/'
xml_folder_path = './raw_data/kaggle/labels/'
save_folder_path = '/generated_data/'

In [6]:
for file_name in tqdm(os.listdir(images_folder_path), desc=f'generating yolo_format_data of {data_type}'):
    img_name = file_name.split('.')[0]
    img_path = images_folder_path + img_name + '.png'
    img = cv2.imread(img_path)
    h_, w_ = img.shape[0], img.shape[1]

    # images 생성
    cv2.imwrite(f'.{save_folder_path}{data_type}/images/train/{img_name}.jpg', img)
    
    xml_file_path = f'{xml_folder_path}{img_name}.xml'
    root = ET.parse(xml_file_path).getroot()
    f = open(f'.{save_folder_path}{data_type}/labels/train/{img_name}.txt', 'w')
    for object_elem in root.findall('.//object'):
        class_type = 0
        xmin_elem  = object_elem.find('./bndbox/xmin')
        ymin_elem  = object_elem.find('./bndbox/ymin')
        xmax_elem  = object_elem.find('./bndbox/xmax')
        ymax_elem  = object_elem.find('./bndbox/ymax')
        
        if xmin_elem is not None and ymin_elem is not None and xmax_elem is not None and ymax_elem is not None:
            
            
            x = round(((int(xmax_elem.text) + int(xmin_elem.text))//2)//w_, 4)
            y = round(((int(ymax_elem.text) + int(ymin_elem.text))//2)//h_, 4)
            w = round((int(xmax_elem.text) - int(xmin_elem.text))/w_, 4)
            h = round((int(ymax_elem.text) - int(ymin_elem.text))/w_, 4)
            text = '{} {} {} {} {}\n'.format(str(class_type), str(x), str(y), str(w), str(h))
            f.write(text)
    f.close()

generating yolo_format_data of kaggle: 100%|████████████████████████████████████████████████| 433/433 [00:05<00:00, 85.35it/s]
